In [1]:
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
import torch
import numpy as np
from simulator import Simulator
import yaml
import pandas as pd

agg_config = yaml.safe_load(open('../configurations/aggregators/fedavg.yaml'))
client_config = yaml.safe_load(open('../configurations/clients/no_poison_no_straggle.yaml'))
data_config = yaml.safe_load(open('../configurations/datasets/mnist_0_1.yaml'))

sim = Simulator(
    **agg_config,
    **client_config,
    **data_config,
)

In [17]:
from aggregators.utils import normalize_weights

normalize_weights(torch.arange(10))

tensor([0.0000, 0.0222, 0.0444, 0.0667, 0.0889, 0.1111, 0.1333, 0.1556, 0.1778,
        0.2000])

In [20]:
list(sim.global_model.model.parameters())[0].device

device(type='cpu')

In [6]:
n_epoch = 100

for epoch in tqdm(
    range(n_epoch), 
    desc=f'Running {str(sim.output_dir)}', 
    leave=True,
):
    picked_clients, to_update_global = sim.step()
    avg_losses = [u.avg_loss for u in to_update_global]
    train_acc_scores = [u.train_acc_score for u in to_update_global]
    test_acc_scores = [u.test_acc_score for u in to_update_global]

    if len(to_update_global):
        # only update the global model if we have any updates
        avg_train_acc = sum(train_acc_scores) / len(train_acc_scores)
        avg_test_acc = sum(test_acc_scores) / len(test_acc_scores)

        # break
    #     # Step 4. Update the global model with the finished local updates
        new_state = sim.aggregator(sim.global_model, to_update_global)

    #     if new_state is not None:
    #         sim.global_model.set_state(new_state)
    # else:
    #     avg_train_acc = 0
    #     avg_test_acc = 0
    
    # pred = sim.global_model.predict(sim.x_test)

Running None:   0%|          | 0/100 [00:00<?, ?it/s]

RuntimeError: stack expects each tensor to be equal size, but got [784] at entry 0 and [] at entry 1

In [11]:
from aggregators.utils import weighted_average
new_global_state = sim.global_model.get_state()
model_weights, update_weights, update_delays = sim.aggregator.parse_updates(to_update_global)


for key, components in zip(new_global_state, zip(*model_weights)):
    print([c.shape for c in components])
    new_global_state[key] = weighted_average(components, update_weights)

    # break
# return new_global_state

[torch.Size([200, 784]), torch.Size([200, 784]), torch.Size([200, 784]), torch.Size([200, 784]), torch.Size([200, 784]), torch.Size([200, 784]), torch.Size([200, 784]), torch.Size([200, 784]), torch.Size([200, 784]), torch.Size([200, 784])]
[torch.Size([200]), torch.Size([200]), torch.Size([200]), torch.Size([200]), torch.Size([200]), torch.Size([200]), torch.Size([200]), torch.Size([200]), torch.Size([200]), torch.Size([200])]
[torch.Size([200, 200]), torch.Size([200, 200]), torch.Size([200, 200]), torch.Size([200, 200]), torch.Size([200, 200]), torch.Size([200, 200]), torch.Size([200, 200]), torch.Size([200, 200]), torch.Size([200, 200]), torch.Size([200, 200])]
[torch.Size([200]), torch.Size([200]), torch.Size([200]), torch.Size([200]), torch.Size([200]), torch.Size([200]), torch.Size([200]), torch.Size([200]), torch.Size([200]), torch.Size([200])]
[torch.Size([2, 200]), torch.Size([2, 200]), torch.Size([2, 200]), torch.Size([2, 200]), torch.Size([2, 200]), torch.Size([2, 200]), tor

In [5]:


model_weights, update_weights, update_delays = sim.aggregator.parse_updates(to_update_global)

weighted_average(model_weights,update_weights)

# w_avg = [
#         # each point has two dimensions, and by stacking them we get 3 dimensions
#         # instead of doing nasty transposing, we can just add matching dims for the weight
#         (
#             torch.stack(p) 
#             * update_weights.reshape([len(model_weights)] + [1] * p[0].ndim)
#         ).sum(0)
#         for p in zip(*model_weights)
#     ]

[tensor([[ -0.3369,  24.1400, -37.0370,  ...,  27.6388,   4.7031,   2.5960],
         [-24.9191, -18.8764, -13.1574,  ..., -25.5334,  -7.5126, -37.7317],
         [-25.3871,  18.7722, -41.9277,  ..., -25.6333,   1.4889,  10.1148],
         ...,
         [ -0.3475, -33.4996, -25.0153,  ...,  -7.8178,  18.8428,  42.9558],
         [ 28.3070, -38.4243, -40.7563,  ..., -37.4338,  -8.5625,  21.0292],
         [ 44.7266,  41.3369,  43.6056,  ...,   6.2606,  10.3698,  18.8232]]),
 tensor([ 27.7479,  28.2241, -42.1896,  18.0164,  24.8717,  36.1386,   3.0739,
          27.7637, -16.3844, -11.1367,  -7.3653, -24.7901, -36.1190,  42.2010,
          27.4532, -17.6969, -10.9478,  37.2056,  31.7967,  28.0664,  34.3884,
         -14.2719, -21.5954,  -7.2163,  16.3790,  44.4648,  -6.9938,  31.8804,
          19.9674,  11.0826,   1.7534, -23.8572,  34.2364,  -3.7545,  32.1327,
         -38.4016, -24.2678,  34.2979,  12.2915, -37.1306,  16.8781,  33.5338,
         -24.6969,  -8.1868,  41.0820, -25.6336,

In [18]:
[w.shape for w in w_avg]

[torch.Size([200, 784]),
 torch.Size([200]),
 torch.Size([200, 200]),
 torch.Size([200]),
 torch.Size([2, 200]),
 torch.Size([2])]

In [3]:
from dataset import load_MNIST, load_CIFAR

x_train,y_train,x_test,y_test = load_CIFAR()

Files already downloaded and verified
Files already downloaded and verified


In [1]:
from models import MLP, ConvNet  

# model = MLP(
#     in_features= 784,
#     out_classes= 2,
#     hidden_layers=[200,200]
# )

model = ConvNet(
    in_features = [32,32],
    out_classes = 2,
    conv_shapes = [16,16],
    kernel_sizes = [3,3],
    hidden_layers = [100,100],
    dropout = 0.1,
)


[1024, 100, 100]


In [4]:
model(x_train[:10])

tensor([[0.5251, 0.4749],
        [0.5019, 0.4981],
        [0.5233, 0.4767],
        [0.5273, 0.4727],
        [0.5284, 0.4716],
        [0.4918, 0.5082],
        [0.5357, 0.4643],
        [0.5172, 0.4828],
        [0.5103, 0.4897],
        [0.5257, 0.4743]], grad_fn=<SoftmaxBackward0>)

In [8]:
model.conv_layers(x_train[:10]).flatten(1).shape

torch.Size([10, 1024])

In [7]:
model.linear_layers

Sequential(
  (0): Sequential(
    (0): Linear(in_features=1024, out_features=100, bias=True)
    (1): ReLU()
  )
  (1): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): ReLU()
  )
  (2): Linear(in_features=100, out_features=2, bias=True)
)